In [20]:
import requests
import pandas as pd

apikey= open('apikey.txt','r').readline()

def get_lat_lng(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address},+UK&key={apikey}"
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        result = data['results'][0]
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']
        return lat, lng
    else:
        return None


In [22]:
OD=pd.read_excel('Heathrow_OD.xlsx').dropna()
OD['Local Auth'] = OD['Local Auth'].astype(str)
vals = OD['Local Auth'].astype(str)


In [23]:
latlist = list()
lnglist = list()
for val in vals:
     curlat, curlng = get_lat_lng(val)
     latlist.append(curlat)
     lnglist.append(curlng)


[57.1498891,
 57.28687230000001,
 50.8348175,
 54.6990638,
 53.0151867,
 56.6979538,
 56.4006214,
 50.82341,
 53.09289279999999,
 51.14510070000001,
 51.81767259999999,
 52.0603616,
 51.5540666,
 51.6569225,
 53.55263,
 54.108967,
 51.57608399999999,
 51.299173,
 53.3186178,
 51.3250102,
 52.1386603,
 51.439933,
 52.4822694,
 52.5707761,
 53.6957522,
 53.81674229999999,
 51.7875779,
 53.231044,
 53.57686469999999,
 52.97893999999999,
 50.7380046,
 51.4153828,
 53.7937996,
 51.880087,
 52.5716157,
 51.5672808,
 51.620475,
 51.504286,
 50.8226288,
 51.4548943,
 52.73911769999999,
 51.4063204,
 52.335589,
 51.74347299999999,
 52.9854591,
 53.7892877,
 53.5933498,
 51.578829,
 53.7247845,
 52.1950788,
 51.5390261,
 52.7183027,
 51.280233,
 51.483707,
 54.892473,
 51.8572309,
 51.5345547,
 52.0029744,
 52.21914289999999,
 55.378051,
 52.7407093,
 51.7355868,
 51.8993855,
 51.8884129,
 53.1610446,
 53.2302974,
 53.235048,
 50.83761000000001,
 51.6606074,
 53.653511,
 55.953252,
 51.5123443,


In [24]:
OD['lat'] = latlist
OD['lng'] = lnglist

,Local Auth,Total Annual Demand,Mode Share Taxi,Mode Share Car,Mode Share Rail,lat,lng
1,Aberdeen City,7000.0,0.284172,0.443765,0.272063,57.149889,-2.093753
2,Aberdeenshire,1000.0,0.000000,1.000000,0.000000,57.286872,-2.381568
3,Adur,13000.0,0.000000,0.887134,0.000000,50.834817,-0.310126
4,Allerdale,3000.0,0.000000,0.218843,0.000000,54.699064,-3.199420
5,Amber Valley,13000.0,0.000000,0.857952,0.062812,53.015187,-1.482307
...,...,...,...,...,...,...,...
358,Wychavon,33000.0,0.352380,0.547394,0.000000,52.152518,-2.033095
359,Wycombe,325000.0,0.423891,0.512929,0.043526,51.638511,-0.807860
360,Wyre,3000.0,0.492094,0.000000,0.507906,53.899610,-2.809301
361,Wyre Forest,17000.0,0.138131,0.809199,0.000000,52.403805,-2.253763
